In [4]:
import requests

# Using Epic Test Server

In [1]:
EPIC_BASE_URL = "https://connectathon.epic.com/interconnect-fhir-oauth/api/FHIR/R4/"

## Scenario 0

In [2]:
DISCOVERY_PATH = ".well-known/smart-configuration"

In [7]:
epic_config = requests.get(EPIC_BASE_URL + DISCOVERY_PATH)

- Payload includes `permission_v2` and `authorize-post` capabilities ✔️
- `code_challenge_methods_supported` includes `S256` ✔️
- `introscpection_endpoint` included ✔️

In [12]:
epic_config.text

'<discovery xmlns="http://hl7.org/fhir"><authorization_endpoint value="https://connectathon.epic.com/Interconnect-Fhir-OAuth/oauth2/authorize" /><token_endpoint value="https://connectathon.epic.com/Interconnect-Fhir-OAuth/oauth2/token" /><token_endpoint_auth_methods_supported value="client_secret_post" /><token_endpoint_auth_methods_supported value="client_secret_basic" /><token_endpoint_auth_methods_supported value="private_key_jwt" /><scopes_supported value="epic.scanning.dmsusername" /><scopes_supported value="fhirUser" /><scopes_supported value="launch" /><scopes_supported value="openid" /><scopes_supported value="profile" /><response_types_supported value="code" /><introspection_endpoint value="https://connectathon.epic.com/Interconnect-Fhir-OAuth/oauth2/introspect" /><capabilities value="launch-ehr" /><capabilities value="launch-standalone" /><capabilities value="client-public" /><capabilities value="client-confidential-symmetric" /><capabilities value="context-banner" /><capabil